In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
import joblib

df = pd.read_csv("backend/schizophrenia_dataset.csv")

df_CHOSEN = df[['Age', 'Gender', 'Education_Level', 'Marital_Status','Occupation', 'Income_Level', 
                'Family_History_of_Schizophrenia','Substance_Use', 'Suicide_Attempts',
                'Social_Support', 'Stress_Factors', 'Medication_Compliance',
                'Diagnosis']]

# Преобразование категориальных данных
categorical_columns = ['Gender', 'Education_Level', 'Marital_Status', 'Occupation', 
                       'Income_Level', 'Family_History_of_Schizophrenia', 
                       'Substance_Use', 'Suicide_Attempts', 'Social_Support', 'Stress_Factors', 'Medication_Compliance']

# Применение LabelEncoder для кодирования категориальных переменных
encoder = LabelEncoder()
for col in categorical_columns:
    df_CHOSEN[col] = encoder.fit_transform(df_CHOSEN[col])

# Разделение на признаки и таргет
X = df_CHOSEN.drop(columns=['Diagnosis'])
y = df_CHOSEN['Diagnosis']

# Разделение на тренировочную и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Применение undersampling для сбалансирования классов
undersampler = RandomUnderSampler(sampling_strategy='majority', random_state=42)
X_train_resampled, y_train_resampled = undersampler.fit_resample(X_train, y_train)

# Используем лучшие гиперпараметры для построения финальной модели
best_model = LogisticRegression(C=0.1, class_weight='balanced', max_iter=500, penalty='l1', solver='saga')

# Обучение модели на сбалансированных данных
best_model.fit(X_train_resampled, y_train_resampled)

# Предсказания
y_pred = best_model.predict(X_test)

# Сохранение модели и энкодеров
joblib.dump(best_model, 'schizophrenia_model.pkl')
joblib.dump(encoder, 'label_encoder.pkl')

/tmp/ipykernel_10311/1323681735.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_CHOSEN[col] = encoder.fit_transform(df_CHOSEN[col])
/tmp/ipykernel_10311/1323681735.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_CHOSEN[col] = encoder.fit_transform(df_CHOSEN[col])
/tmp/ipykernel_10311/1323681735.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https:

['label_encoder.pkl']

In [3]:
print("Accuracy: ", accuracy_score(y_test,y_pred))
print("Classification report: ", classification_report(y_test, y_pred))
print("Confusion matrix: ", confusion_matrix(y_test,y_pred))

Accuracy:  0.826
Classification report:                precision    recall  f1-score   support

           0       0.88      0.87      0.88      1406
           1       0.70      0.72      0.71       594

    accuracy                           0.83      2000
   macro avg       0.79      0.80      0.79      2000
weighted avg       0.83      0.83      0.83      2000

Confusion matrix:  [[1222  184]
 [ 164  430]]
